In [1]:
import cv2
import numpy as np
import torch
from pathlib import Path

In [2]:
import os

# 이미지와 바운딩 박스 디렉토리 설정
image_dir = "C:/Users/wjdrl/OneDrive - dgu.ac.kr/DA/BAF/project/2024_2/code/Crawling2-2/train/images"
bbox_dir = "C:/Users/wjdrl/OneDrive - dgu.ac.kr/DA/BAF/project/2024_2/code/Crawling2-2/train/labels"

# 실험 파일 설정
image_file = "image_1_jpg.rf.03dc7caa804ba5ddf4d0464d761ccec6.jpg"
bbox_file = "image_1_jpg.rf.03dc7caa804ba5ddf4d0464d761ccec6.txt"

# 이미지와 바운딩 박스 경로 설정
image_path = os.path.join(image_dir, image_file).replace("\\", "/")
bbox_path = os.path.join(bbox_dir, bbox_file).replace("\\", "/")

In [3]:
image_path

'C:/Users/wjdrl/OneDrive - dgu.ac.kr/DA/BAF/project/2024_2/code/Crawling2-2/train/images/image_1_jpg.rf.03dc7caa804ba5ddf4d0464d761ccec6.jpg'

In [4]:
img_width = 640             # 사진의 너비
img_height = 640            # 사진의 높이

def extract_object(image, bbox, img_width, img_height):
    x_center, y_center, width, height = bbox  # YOLOv5에서 반환된 바운딩 박스 좌표를 픽셀 단위로 변환
    
    # 정규화된 좌표를 이미지 크기에 맞게 변환
    x_center, y_center = int(x_center * img_width), int(y_center * img_height)
    width, height = int(width * img_width), int(height * img_height)
    
    # 좌표를 이미지 크기 안에서 잘라냄
    x1 = max(0, x_center - width // 2)
    y1 = max(0, y_center - height // 2)
    x2 = min(img_width, x_center + width // 2)
    y2 = min(img_height, y_center + height // 2)
    
    # 잘린 좌표가 이미지 크기를 벗어나면 경고 출력
    if x1 >= x2 or y1 >= y2:
        print(f"Warning: Bounding box is out of bounds for image. Using default empty image.")
        return np.zeros((100, 100, 3), dtype=np.uint8)  # 기본값으로 빈 이미지 반환
    
    # 이미지에서 객체 영역을 잘라냄
    cropped_image = image[y1:y2, x1:x2]
    
    return cropped_image
# YOLOv5 모델 추론 후 결과 필터링
def process_detections(image, detections, img_width, img_height):
    top_images = []
    bottom_images = []
    accessory_images = []

    for det in detections:
        # detections의 형식: [class, x_center, y_center, width, height, confidence]
        class_id, bbox = int(det[0]), det[1:5]
        
        if class_id == 0:  # 상의
            top_images.append(extract_object(image, bbox, img_width, img_height))
        elif class_id == 1:  # 하의
            bottom_images.append(extract_object(image, bbox, img_width, img_height))
        elif class_id == 2:  # 악세사리
            accessory_images.append(extract_object(image, bbox, img_width, img_height))
    
    return top_images, bottom_images, accessory_images

In [5]:
def image_to_rgb_vectors(image):
    if image is None or image.size == 0:
        return None, None, None  # 이미지가 비어 있으면 None 반환

    # 이미지 크기를 100x100으로 고정
    image_resized = cv2.resize(image, (100, 100))
    
    # 각 채널(R, G, B) 분리
    r_channel = image_resized[:, :, 0].flatten()  # R 채널을 1차원 벡터로 변환
    g_channel = image_resized[:, :, 1].flatten()  # G 채널을 1차원 벡터로 변환
    b_channel = image_resized[:, :, 2].flatten()  # B 채널을 1차원 벡터로 변환

    return r_channel, g_channel, b_channel


In [6]:
# detections => 우리가 라벨링한 파일 = bbox
def load_detections_from_label_file(label_file_path):
    detections = []
    with open(label_file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            detections.append([class_id, x_center, y_center, width, height])
    return detections

# 예시 라벨 파일 경로
label_file_path = bbox_path

# 라벨 파일에서 `detections` 정보를 불러오기
detections = load_detections_from_label_file(label_file_path)


# 감지된 객체들을 벡터화하여 각 세트에 저장
def process_image_sets(image, detections, img_width, img_height):
    # 감지된 객체 분류
    top_images, bottom_images, accessory_images = process_detections(image, detections, img_width, img_height)

    # 벡터 저장을 위한 딕셔너리 초기화
    image_sets = {
        'upper_body': [image_to_rgb_vectors(img) for img in top_images],
        'lower_body': [image_to_rgb_vectors(img) for img in bottom_images],
        'accessory': [image_to_rgb_vectors(img) for img in accessory_images]
    }
    
    return image_sets

# 각 세트의 상의, 하의, 악세사리 벡터를 저장할 딕셔너리
image_sets = {}

# 예시: 여러 세트의 상의, 하의, 악세사리 이미지 파일 경로
sets = [
    {'upper_body': 'upper_body_image_1.jpg', 'lower_body': 'lower_body_image_1.jpg', 'accessory': 'accessory_image_1.jpg'},
    {'upper_body': 'upper_body_image_2.jpg', 'lower_body': 'lower_body_image_2.jpg', 'accessory': 'accessory_image_2.jpg'},
    # 추가 이미지 세트
]

In [7]:
# 테스트 이미지와 감지 결과로부터 상의, 하의, 악세사리 벡터 생성
image_file = image_path
image = cv2.imread(image_file)
img_height, img_width = image.shape[:2]
image_sets = process_image_sets(image, detections, img_width, img_height)

# 결과 확인
print(image_sets)

{'upper_body': [(array([238, 238, 238, ...,   7,  10,  14], dtype=uint8), array([239, 239, 239, ...,   7,  10,  14], dtype=uint8), array([237, 237, 237, ...,   7,  10,  14], dtype=uint8))], 'lower_body': [(array([209, 214, 200, ..., 127, 127, 118], dtype=uint8), array([202, 207, 193, ..., 115, 117, 112], dtype=uint8), array([199, 205, 191, ..., 101, 106, 102], dtype=uint8))], 'accessory': [(array([ 99, 100, 104, ...,  97, 102, 103], dtype=uint8), array([ 96,  97, 101, ..., 111, 116, 117], dtype=uint8), array([ 92,  93,  97, ..., 111, 115, 116], dtype=uint8)), (array([111, 117, 125, ..., 174, 172, 171], dtype=uint8), array([112, 117, 123, ..., 191, 189, 188], dtype=uint8), array([103, 108, 113, ..., 188, 186, 185], dtype=uint8))]}


In [8]:
# 예시(원본)
# # 모든 이미지와 라벨 파일을 처리하여 저장할 딕셔너리
# all_image_sets = {}

# # 이미지 파일과 라벨 파일 목록 불러오기
# image_files = os.listdir(image_dir)
# label_files = os.listdir(label_dir)

# # 각 이미지와 해당 라벨 파일을 처리
# for image_file in image_files:
#     # 이미지 경로 및 파일명 설정
#     image_path = os.path.join(image_dir, image_file)
#     label_file = image_file.replace('.jpg', '.txt')  # 이미지 파일명과 라벨 파일명 맞추기
#     label_path = os.path.join(label_dir, label_file)

#     # 이미지와 라벨 파일이 모두 존재할 때만 처리
#     if os.path.exists(label_path):
#         # 이미지 로드 및 크기 확인
#         image = cv2.imread(image_path)
#         if image is None:
#             print(f"Error: Could not load image {image_path}")
#             continue
        
#         img_height, img_width = image.shape[:2]

#         # 라벨 파일에서 바운딩 박스 정보 로드
#         detections = load_detections_from_label_file(label_path)

#         # 각 이미지의 객체를 벡터화하여 저장
#         image_vectors = process_image_sets(image, detections, img_width, img_height)

#         # 고유 키를 파일명으로 설정하여 딕셔너리에 저장  => 가독성이 떨어지긴 하는데
#         all_image_sets[image_file] = image_vectors

# # 전체 결과 확인
# for image_name, vectors in all_image_sets.items():
#     print(f"Image: {image_name}")
#     print("Upper Body Vectors:", vectors['upper_body'])
#     print("Lower Body Vectors:", vectors['lower_body'])
#     print("Accessory Vectors:", vectors['accessory'])
#     print("\n")

##### train

In [23]:
image_dir = "./Crawling2-2/train/images"
label_dir = "./Crawling2-2/train/labels"

# 모든 이미지와 라벨 파일을 처리하여 저장할 딕셔너리
train_image_sets = {}

# 이미지 파일과 라벨 파일 목록 불러오기
image_files = os.listdir(image_dir)
label_files = os.listdir(label_dir)

# 각 이미지와 해당 라벨 파일을 처리
for image_file in image_files:
    # 이미지 경로 및 파일명 설정
    image_path = os.path.join(image_dir, image_file)
    label_file = image_file.replace('.jpg', '.txt')  # 이미지 파일명과 라벨 파일명 맞추기
    label_path = os.path.join(label_dir, label_file)

    # 이미지와 라벨 파일이 모두 존재할 때만 처리
    if os.path.exists(label_path):
        # 이미지 로드 및 크기 확인
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue
        
        img_height, img_width = image.shape[:2]

        # 라벨 파일에서 바운딩 박스 정보 로드
        detections = load_detections_from_label_file(label_path)

        # 각 이미지의 객체를 벡터화하여 저장
        image_vectors = process_image_sets(image, detections, img_width, img_height)

        # 고유 키를 파일명으로 설정하여 딕셔너리에 저장  => 가독성이 떨어지긴 하는데
        train_image_sets[image_file] = image_vectors

# 전체 결과 확인
for image_name, vectors in train_image_sets.items():
    print(f"Image: {image_name}")
    print("Upper Body Vectors:", vectors['upper_body'])
    print("Lower Body Vectors:", vectors['lower_body'])
    print("Accessory Vectors:", vectors['accessory'])
    print("\n")

Image: image_1001_jpg.rf.7e10c26e11be6c0f91af1cc4b6939314.jpg
Upper Body Vectors: [(array([169, 168, 171, ..., 152, 148, 149], dtype=uint8), array([197, 197, 198, ..., 176, 175, 175], dtype=uint8), array([202, 202, 195, ..., 196, 194, 197], dtype=uint8))]
Lower Body Vectors: [(array([168, 167, 160, ..., 124, 135, 137], dtype=uint8), array([153, 153, 144, ..., 107, 119, 126], dtype=uint8), array([137, 135, 127, ...,  83,  99, 111], dtype=uint8))]
Accessory Vectors: [(array([144, 144, 145, ...,  19,  19,  19], dtype=uint8), array([149, 149, 150, ...,  20,  20,  20], dtype=uint8), array([148, 148, 149, ...,  16,  17,  18], dtype=uint8)), (array([120, 117, 113, ..., 166, 166, 167], dtype=uint8), array([100,  98,  96, ..., 151, 150, 150], dtype=uint8), array([ 75,  73,  70, ..., 132, 131, 131], dtype=uint8))]


Image: image_1002_jpg.rf.ce2d9d012308af8427470a4e2da5718a.jpg
Upper Body Vectors: [(array([69, 52, 42, ..., 32, 35, 37], dtype=uint8), array([72, 55, 45, ..., 35, 39, 42], dtype=uint

In [24]:
len(train_image_sets.items()) # 길이같음

755

In [25]:
# 데이터 준비
upper_body_data_train = []  # 상의 벡터 (고정 크기 30000)
lower_body_label_train = []  # 하의 벡터 (고정 크기 30000)
accessory_data_train = []  # 가변 길이 악세사리 시퀀스
accessory_labels_train = []  # 가변 길이 악세사리 레이블 시퀀스

for image_name, vectors in train_image_sets.items():
    # 상의 벡터 처리 (R, G, B 결합)
    if len(vectors['upper_body']) > 0:
        r_vector, g_vector, b_vector = vectors['upper_body'][0]
        upper_body_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
    else:
        upper_body_vector = np.zeros(30000)  # 기본값

    upper_body_data_train.append(upper_body_vector)

    # 하의 벡터 처리 (R, G, B 결합)
    if len(vectors['lower_body']) > 0:
        r_vector, g_vector, b_vector = vectors['lower_body'][0]
        lower_body_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
    else:
        lower_body_vector = np.zeros(30000)  # 기본값

    lower_body_label_train.append(lower_body_vector)

    # 악세사리 벡터 처리 (R, G, B 결합 및 시퀀스 형태로 저장)
    accessory_sequence = []
    accessory_label_sequence = []
    for accessory in vectors['accessory']:
        r_vector, g_vector, b_vector = accessory
        accessory_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
        accessory_sequence.append(accessory_vector)
        accessory_label_sequence.append(accessory_vector)  # 동일한 벡터를 레이블로 사용

    accessory_data_train.append(accessory_sequence)
    accessory_labels_train.append(accessory_label_sequence)

# numpy 배열로 변환
upper_body_data_train = np.array(upper_body_data_train)
lower_body_label_train = np.array(lower_body_label_train)
accessory_data_train = np.array([np.array(seq) for seq in accessory_data_train], dtype=object)  # 가변 길이 시퀀스
accessory_labels_train = np.array([np.array(seq) for seq in accessory_labels_train], dtype=object)  # 가변 길이 시퀀스

In [26]:
# 데이터 개수
print("Length of upper_body_data_train:", len(upper_body_data_train))
print("Length of lower_body_label_train:", len(lower_body_label_train))
print("Length of accessory_data_train:", len(accessory_data_train))
print("Length of accessory_labels_train:", len(accessory_labels_train))

Length of upper_body_data_train: 755
Length of lower_body_label_train: 755
Length of accessory_data_train: 755
Length of accessory_labels_train: 755


##### valid

In [27]:
image_dir = "./Crawling2-2/valid/images"
label_dir = "./Crawling2-2/valid/labels"

# 모든 이미지와 라벨 파일을 처리하여 저장할 딕셔너리
valid_image_sets = {}

# 이미지 파일과 라벨 파일 목록 불러오기
image_files = os.listdir(image_dir)
label_files = os.listdir(label_dir)

# 각 이미지와 해당 라벨 파일을 처리
for image_file in image_files:
    # 이미지 경로 및 파일명 설정
    image_path = os.path.join(image_dir, image_file)
    label_file = image_file.replace('.jpg', '.txt')  # 이미지 파일명과 라벨 파일명 맞추기
    label_path = os.path.join(label_dir, label_file)

    # 이미지와 라벨 파일이 모두 존재할 때만 처리
    if os.path.exists(label_path):
        # 이미지 로드 및 크기 확인
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue
        
        img_height, img_width = image.shape[:2]

        # 라벨 파일에서 바운딩 박스 정보 로드
        detections = load_detections_from_label_file(label_path)

        # 각 이미지의 객체를 벡터화하여 저장
        image_vectors = process_image_sets(image, detections, img_width, img_height)

        # 고유 키를 파일명으로 설정하여 딕셔너리에 저장  => 가독성이 떨어지긴 하는데
        valid_image_sets[image_file] = image_vectors

# 전체 결과 확인
for image_name, vectors in valid_image_sets.items():
    print(f"Image: {image_name}")
    print("Upper Body Vectors:", vectors['upper_body'])
    print("Lower Body Vectors:", vectors['lower_body'])
    print("Accessory Vectors:", vectors['accessory'])
    print("\n")

Image: image_0_jpg.rf.fc1f0218d9d8669adc1b06b5d3921abc.jpg
Upper Body Vectors: [(array([182, 182, 182, ..., 182, 187, 184], dtype=uint8), array([187, 187, 187, ..., 184, 189, 186], dtype=uint8), array([188, 188, 188, ..., 185, 190, 187], dtype=uint8))]
Lower Body Vectors: [(array([127,  25,  20, ..., 175, 175, 175], dtype=uint8), array([131,  29,  25, ..., 177, 177, 177], dtype=uint8), array([128,  27,  23, ..., 177, 177, 177], dtype=uint8))]
Accessory Vectors: [(array([157, 157, 157, ..., 197, 197, 196], dtype=uint8), array([159, 159, 159, ..., 192, 192, 191], dtype=uint8), array([159, 159, 159, ..., 193, 193, 192], dtype=uint8)), (array([ 24,  30,  32, ..., 188, 188, 188], dtype=uint8), array([ 22,  29,  30, ..., 188, 188, 188], dtype=uint8), array([ 22,  29,  30, ..., 188, 188, 188], dtype=uint8))]


Image: image_1007_jpg.rf.05b685c76d824be326e6d2a8c3eab956.jpg
Upper Body Vectors: [(array([102, 102, 100, ..., 101,  89,  89], dtype=uint8), array([114, 114, 112, ..., 110,  97,  97], d

In [28]:
# 데이터 준비
upper_body_data_valid = []  # 상의 벡터 (고정 크기 30000)
lower_body_label_valid = []  # 하의 벡터 (고정 크기 30000)
accessory_data_valid = []  # 가변 길이 악세사리 시퀀스
accessory_labels_valid = []  # 가변 길이 악세사리 레이블 시퀀스

for image_name, vectors in valid_image_sets.items():
    # 상의 벡터 처리 (R, G, B 결합)
    if len(vectors['upper_body']) > 0:
        r_vector, g_vector, b_vector = vectors['upper_body'][0]
        upper_body_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
    else:
        upper_body_vector = np.zeros(30000)  # 기본값

    upper_body_data_valid.append(upper_body_vector)

    # 하의 벡터 처리 (R, G, B 결합)
    if len(vectors['lower_body']) > 0:
        r_vector, g_vector, b_vector = vectors['lower_body'][0]
        lower_body_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
    else:
        lower_body_vector = np.zeros(30000)  # 기본값

    lower_body_label_valid.append(lower_body_vector)

    # 악세사리 벡터 처리 (R, G, B 결합 및 시퀀스 형태로 저장)
    accessory_sequence = []
    accessory_label_sequence = []
    for accessory in vectors['accessory']:
        r_vector, g_vector, b_vector = accessory
        accessory_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
        accessory_sequence.append(accessory_vector)
        accessory_label_sequence.append(accessory_vector)  # 동일한 벡터를 레이블로 사용

    accessory_data_valid.append(accessory_sequence)
    accessory_labels_valid.append(accessory_label_sequence)

# numpy 배열로 변환
upper_body_data_valid = np.array(upper_body_data_valid)
lower_body_label_valid = np.array(lower_body_label_valid)
accessory_data_valid = np.array([np.array(seq) for seq in accessory_data_valid], dtype=object)  # 가변 길이 시퀀스
accessory_labels_valid = np.array([np.array(seq) for seq in accessory_labels_valid], dtype=object)  # 가변 길이 시퀀스

##### test

In [29]:
image_dir = "./Crawling2-2/test/images"
label_dir = "./Crawling2-2/test/labels"

# 모든 이미지와 라벨 파일을 처리하여 저장할 딕셔너리
test_image_sets = {}

# 이미지 파일과 라벨 파일 목록 불러오기
image_files = os.listdir(image_dir)
label_files = os.listdir(label_dir)

# 각 이미지와 해당 라벨 파일을 처리
for image_file in image_files:
    # 이미지 경로 및 파일명 설정
    image_path = os.path.join(image_dir, image_file)
    label_file = image_file.replace('.jpg', '.txt')  # 이미지 파일명과 라벨 파일명 맞추기
    label_path = os.path.join(label_dir, label_file)

    # 이미지와 라벨 파일이 모두 존재할 때만 처리
    if os.path.exists(label_path):
        # 이미지 로드 및 크기 확인
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue
        
        img_height, img_width = image.shape[:2]

        # 라벨 파일에서 바운딩 박스 정보 로드
        detections = load_detections_from_label_file(label_path)

        # 각 이미지의 객체를 벡터화하여 저장
        image_vectors = process_image_sets(image, detections, img_width, img_height)

        # 고유 키를 파일명으로 설정하여 딕셔너리에 저장  => 가독성이 떨어지긴 하는데
        test_image_sets[image_file] = image_vectors

# 전체 결과 확인
for image_name, vectors in test_image_sets.items():
    print(f"Image: {image_name}")
    print("Upper Body Vectors:", vectors['upper_body'])
    print("Lower Body Vectors:", vectors['lower_body'])
    print("Accessory Vectors:", vectors['accessory'])
    print("\n")

Image: image_1000_jpg.rf.335ab735f197afc717f5629c35a6e988.jpg
Upper Body Vectors: [(array([191, 192, 185, ...,  33,  63, 166], dtype=uint8), array([188, 186, 176, ...,  18,  50, 154], dtype=uint8), array([186, 179, 166, ...,  16,  48, 152], dtype=uint8))]
Lower Body Vectors: [(array([246, 243, 252, ..., 155, 142, 142], dtype=uint8), array([239, 239, 248, ..., 150, 138, 138], dtype=uint8), array([224, 220, 229, ..., 151, 139, 140], dtype=uint8))]
Accessory Vectors: [(array([133, 133, 133, ..., 106, 106, 106], dtype=uint8), array([136, 137, 138, ..., 105, 105, 105], dtype=uint8), array([140, 141, 141, ..., 109, 109, 109], dtype=uint8)), (array([120, 116, 114, ..., 100,  96,  98], dtype=uint8), array([118, 114, 110, ...,  91,  86,  88], dtype=uint8), array([118, 114, 110, ...,  95,  92,  94], dtype=uint8))]


Image: image_1004_jpg.rf.e164a7f47644a53f2800021b384674eb.jpg
Upper Body Vectors: [(array([ 52,  61,  66, ..., 113, 113, 111], dtype=uint8), array([ 66,  77,  85, ..., 115, 112, 112]

In [30]:
# 데이터 준비
upper_body_data = []  # 상의 벡터 (고정 크기 30000)
lower_body_labels = []  # 하의 벡터 (고정 크기 30000)
accessory_data = []  # 가변 길이 악세사리 시퀀스
accessory_labels = []  # 가변 길이 악세사리 레이블 시퀀스

for image_name, vectors in valid_image_sets.items():
    # 상의 벡터 처리 (R, G, B 결합)
    if len(vectors['upper_body']) > 0:
        r_vector, g_vector, b_vector = vectors['upper_body'][0]
        upper_body_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
    else:
        upper_body_vector = np.zeros(30000)  # 기본값

    upper_body_data.append(upper_body_vector)

    # 하의 벡터 처리 (R, G, B 결합)
    if len(vectors['lower_body']) > 0:
        r_vector, g_vector, b_vector = vectors['lower_body'][0]
        lower_body_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
    else:
        lower_body_vector = np.zeros(30000)  # 기본값

    lower_body_labels.append(lower_body_vector)

    # 악세사리 벡터 처리 (R, G, B 결합 및 시퀀스 형태로 저장)
    accessory_sequence = []
    accessory_label_sequence = []
    for accessory in vectors['accessory']:
        r_vector, g_vector, b_vector = accessory
        accessory_vector = np.concatenate([r_vector, g_vector, b_vector])  # 크기: 30000
        accessory_sequence.append(accessory_vector)
        accessory_label_sequence.append(accessory_vector)  # 동일한 벡터를 레이블로 사용

    accessory_data.append(accessory_sequence)
    accessory_labels.append(accessory_label_sequence)

# numpy 배열로 변환
upper_body_data = np.array(upper_body_data)
lower_body_labels = np.array(lower_body_labels)
accessory_data = np.array([np.array(seq) for seq in accessory_data], dtype=object)  # 가변 길이 시퀀스
accessory_labels = np.array([np.array(seq) for seq in accessory_labels], dtype=object)  # 가변 길이 시퀀스



#### model

##### Fully Connected Neural Network

In [31]:
# from tensorflow.keras import layers, models

# # 딥러닝 모델 정의
# model = models.Sequential([
#     layers.Dense(512, activation='relu', input_shape=(30000,)),  # 상의의 1차원 벡터 입력 (100x100x3)
#     layers.Dense(256, activation='relu'),
#     layers.Dense(128, activation='relu'),
#     layers.Dense(30000, activation='sigmoid')  # 하의의 1차원 벡터를 출력 (100x100x3)
# ])

# # 모델 컴파일
# model.compile(optimizer='adam',
#               loss='mean_squared_error',  # 이미지 복원과 비슷하게 MSE 사용
#               metrics=['accuracy'])

# # train_data와 train_labels 준비
# train_data = []
# train_labels = []

# for vectors in image_sets.values():
#     # 상의 벡터 가져오기
#     if len(vectors) > 0:
#         r_vector, g_vector, b_vector = vectors[0]  # 상의 벡터: 첫 번째 요소
#         upper_body_vector = np.concatenate([r_vector, g_vector, b_vector])
#     else:
#         upper_body_vector = np.zeros(30000)

#     # 하의 벡터 가져오기
#     if len(vectors) > 1:
#         r_vector, g_vector, b_vector = vectors[1]  # 하의 벡터: 두 번째 요소
#         lower_body_vector = np.concatenate([r_vector, g_vector, b_vector])
#     else:
#         lower_body_vector = np.zeros(30000)

#     # 악세사리 벡터 가져오기
#     if len(vectors) > 2:
#         r_vector, g_vector, b_vector = vectors[2]  # 악세사리 벡터: 세 번째 요소
#         accessory_vector = np.concatenate([r_vector, g_vector, b_vector])
#     else:
#         accessory_vector = np.zeros(30000)

#     # 상의 벡터를 train_data에 추가
#     train_data.append(upper_body_vector)
#     # 하의와 악세사리 벡터를 합산하여 train_labels에 추가
#     train_labels.append(lower_body_vector + accessory_vector)

# # numpy 배열로 변환
# train_data = np.array(train_data)
# train_labels = np.array(train_labels)

# # 모델 학습
# model.fit(train_data, train_labels, epochs=10, batch_size=32)

# # 학습 후 모델 저장
# model.save('pattern_color_model.h5')

In [32]:
# from tensorflow.keras.models import load_model

# # 학습된 모델 로드
# model = load_model('pattern_color_model.h5')

# # valid_image_sets에서 상의 벡터를 가져와 numpy 배열로 변환
# # 각 upper_body 벡터를 R, G, B로 분리한 후 결합
# valid_data = []
# for vectors in valid_image_sets.values():
#     if len(vectors['upper_body']) > 0:
#         r_vector, g_vector, b_vector = vectors['upper_body'][0]
#         upper_body_vector = np.concatenate([r_vector, g_vector, b_vector])
#         valid_data.append(upper_body_vector)

# # numpy 배열로 변환
# valid_data = np.array(valid_data)

# # 모델 예측
# predicted_vectors = model.predict(valid_data)

# # 예측 결과 처리 및 이미지 저장
# output_dir = "predictions"
# os.makedirs(output_dir, exist_ok=True)  # 폴더가 없으면 생성

# for idx, (image_name, vectors) in enumerate(valid_image_sets.items()):
#     # 예측된 벡터 분리
#     predicted_combined_vector = predicted_vectors[idx]
#     predicted_lower_body_vector = predicted_combined_vector[:30000]  # 첫 30000은 하의 벡터
#     predicted_accessory_vector = predicted_combined_vector[30000:]   # 나머지 30000은 악세사리 벡터

#     # 예외 처리: 벡터가 비어 있는 경우 기본값으로 채우기
#     if predicted_lower_body_vector.size == 0:
#         predicted_lower_body_vector = np.zeros(30000)
#     if predicted_accessory_vector.size == 0:
#         predicted_accessory_vector = np.zeros(30000)

#     # 예측된 벡터를 이미지로 변환
#     predicted_lower_body_image = predicted_lower_body_vector.reshape(100, 100, 3)
#     predicted_lower_body_image = np.clip(predicted_lower_body_image, 0, 255).astype('uint8')

#     predicted_accessory_image = predicted_accessory_vector.reshape(100, 100, 3)
#     predicted_accessory_image = np.clip(predicted_accessory_image, 0, 255).astype('uint8')

#     # 예측된 이미지 저장
#     lower_body_image_path = os.path.join(output_dir, f"predicted_lower_body_{image_name}.jpg")
#     accessory_image_path = os.path.join(output_dir, f"predicted_accessory_{image_name}.jpg")

#     # 이미지 저장
#     cv2.imwrite(lower_body_image_path, predicted_lower_body_image)
#     cv2.imwrite(accessory_image_path, predicted_accessory_image)

#     print(f"Predicted images saved: '{lower_body_image_path}' and '{accessory_image_path}'")

##### CNN+LSTM

상의 -> 하의  
악세사리 -> 악세사리

In [33]:
from tensorflow.keras import layers, models, Input
import numpy as np

# 최대 시퀀스 길이를 5로 설정
MAX_SEQUENCE_LENGTH = 5

# 패딩 함수 => 만약 우리가 3개의 output을 냈다면 뒤의 2개는 0 벡터로 표현함
def pad_sequence(sequence, max_length, vector_size):
    # 현재 시퀀스 길이가 max_length보다 작으면 0 벡터를 추가
    while len(sequence) < max_length:
        sequence.append(np.zeros(vector_size))
    return np.array(sequence)

# 모델 구성
# 상의 벡터 입력 (고정 크기: 30000)
upper_body_input = Input(shape=(30000,))
x = layers.Dense(512, activation='relu')(upper_body_input)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)

# 하의 벡터 출력 (고정 크기: 30000)
lower_body_output = layers.Dense(30000, activation='sigmoid', name='lower_body_output')(x)

# 악세사리 벡터 입력 (고정된 최대 길이: MAX_SEQUENCE_LENGTH, 각 벡터 크기 30000)
accessory_input = Input(shape=(MAX_SEQUENCE_LENGTH, 30000))
accessory_rnn = layers.LSTM(128, activation='relu', return_sequences=True)(accessory_input)

# 상의 벡터를 악세사리 시퀀스 길이에 맞게 확장
x_repeated = layers.RepeatVector(MAX_SEQUENCE_LENGTH)(x)

# 상의 벡터와 악세사리 RNN 출력 결합
combined = layers.Concatenate(axis=-1)([x_repeated, accessory_rnn])
accessory_output = layers.TimeDistributed(layers.Dense(30000, activation='sigmoid'))(combined)

# 모델 정의 (입력: 상의 + 악세사리, 출력: 하의 + 악세사리)
model = models.Model(inputs=[upper_body_input, accessory_input], outputs=[lower_body_output, accessory_output])

# 모델 컴파일
model.compile(
    optimizer='adam',
    loss=['mean_squared_error', 'mean_squared_error'],
    metrics=[['accuracy'], ['accuracy']]  # 각 출력에 대해 메트릭을 지정
)

# 모델 요약 출력
model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 30000)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 512)       │ 15,360,512 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256)       │    131,328 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     32,896 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 5, 30000)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_1     │ (None, 5, 128)    │          0 │ dense_6[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 5, 128)    │ 15,426,048 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 5, 256)    │          0 │ repeat_vector_1[… │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lower_body_output   │ (None, 30000)     │  3,870,000 │ dense_6[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 5, 30000)  │  7,710,000 │ concatenate_1[0]… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 42,530,784 (162.24 MB)

 Trainable params: 42,530,784 (162.24 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
import numpy as np

# 벡터 크기와 시퀀스 길이 설정
VECTOR_SIZE = 30000
MAX_SEQUENCE_LENGTH = 5

# 패딩 함수
def pad_sequence(sequence, max_length, vector_size):
    padded_sequence = list(sequence)[:max_length]  # 최대 길이까지 잘라내기
    while len(padded_sequence) < max_length:
        padded_sequence.append(np.zeros(vector_size))  # 0 벡터로 패딩
    return np.array(padded_sequence)

# 악세사리 데이터를 패딩하여 3D 형태로 변환
accessory_data_train_padded = np.array(
    [pad_sequence(seq, MAX_SEQUENCE_LENGTH, VECTOR_SIZE) for seq in accessory_data_train]
)
accessory_labels_train_padded = np.array(
    [pad_sequence(seq, MAX_SEQUENCE_LENGTH, VECTOR_SIZE) for seq in accessory_labels_train]
)

# 데이터 준비 확인
print("Shape of accessory_data_train_padded:", accessory_data_train_padded.shape)
print("Shape of accessory_labels_train_padded:", accessory_labels_train_padded.shape)

# 모델 학습
history = model.fit(
    [upper_body_data_train, accessory_data_train_padded],  # 입력 데이터
    [lower_body_label_train, accessory_labels_train_padded],  # 출력 데이터 (레이블)
    epochs=5,
    batch_size=32
)

Shape of accessory_data_train_padded: (755, 5, 30000)
Shape of accessory_labels_train_padded: (755, 5, 30000)
Epoch 1/5


c:\Users\wjdrl\anaconda3\envs\jsy\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_20', 'keras_tensor_25']. Received: the structure of inputs=('*', '*')
  warnings.warn(


24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 170ms/step - loss: 19154.7227 - lower_body_output_accuracy: 0.0069 - lower_body_output_loss: 10899.7822 - time_distributed_1_accuracy: 0.5352 - time_distributed_1_loss: 8255.0293
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - loss: 19802.2324 - lower_body_output_accuracy: 0.0049 - lower_body_output_loss: 11415.8652 - time_distributed_1_accuracy: 0.5804 - time_distributed_1_loss: 8384.2646
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - loss: 19790.0000 - lower_body_output_accuracy: 0.0065 - lower_body_output_loss: 11820.3125 - time_distributed_1_accuracy: 0.5930 - time_distributed_1_loss: 7977.4170
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 173ms/step - loss: 19484.9277 - lower_body_output_accuracy: 0.0092 - lower_body_output_loss: 11562.0566 - time_distributed_1_accuracy: 0.5842 - time_distributed_1_loss: 7913.8560
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 167ms/step - loss: 19999.0820 - lower_body_output_accuracy: 0.0101 - lower_body_output_loss: 1

In [35]:
# # 악세사리 데이터를 패딩하여 3D 형태로 변환
# accessory_data_valid_padded = np.array(
#     [pad_sequence(seq, MAX_SEQUENCE_LENGTH, VECTOR_SIZE) for seq in accessory_data_valid]
# )
# accessory_labels_train_padded = np.array(
#     [pad_sequence(seq, MAX_SEQUENCE_LENGTH, VECTOR_SIZE) for seq in accessory_labels_valid]
# )

# # 모델 예측
# predictions = model.predict([upper_body_data_valid, accessory_data_valid_padded])

# # 반환된 값의 형태와 개수 확인
# print("Type of predictions:", type(predictions))
# if isinstance(predictions, (list, tuple)):
#     print("Number of predictions:", len(predictions))
#     for idx, pred in enumerate(predictions):
#         print(f"Shape of predictions[{idx}]:", pred.shape)
# else:
#     print("Shape of predictions:", predictions.shape)

In [36]:
# import os
# import numpy as np
# import cv2
# from tensorflow.keras.models import load_model

# # 학습된 모델 로드
# model = load_model('pattern_color_model.h5')

# # 저장할 폴더 경로 설정
# output_folder = 'predicted_images'
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)  # 폴더가 없으면 생성

# # 유효성 검사용 데이터 준비
# upper_body_data_valid = np.array(upper_body_data_valid)
# accessory_data_valid_padded = np.array(
#     [pad_sequence(seq, MAX_SEQUENCE_LENGTH, 30000) for seq in accessory_data_valid], dtype=np.float32
# )

# # 모델 예측
# predictions = model.predict([upper_body_data_valid, accessory_data_valid_padded])

# # 예측값을 확인하여 처리
# if isinstance(predictions, list) and len(predictions) == 2:
#     lower_body_pred = predictions[0]
#     accessory_pred = predictions[1]
# else:
#     print("Error: 예측 결과가 예상한 형태가 아닙니다.")
#     lower_body_pred = np.array([])
#     accessory_pred = np.array([])

# # 예측 벡터 크기 확인
# print("Shape of lower_body_pred:", lower_body_pred.shape)
# print("Shape of accessory_pred:", accessory_pred.shape)

# # 예측 결과 처리 및 이미지 저장
# if lower_body_pred.size > 0 and accessory_pred.size > 0:
#     for i in range(min(5, len(lower_body_pred))):  # 처음 5개만 저장
#         # 하의 예측 벡터 처리
#         if lower_body_pred[i].size == 30000:  # 벡터 크기가 30000인지 확인
#             predicted_lower_body_image = lower_body_pred[i].reshape(100, 100, 3)
#             predicted_lower_body_image = np.clip(predicted_lower_body_image, 0, 255).astype('uint8')
#             cv2.imwrite(os.path.join(output_folder, f"predicted_lower_body_image_{i}.jpg"), predicted_lower_body_image)
#             print(f"Saved: predicted_lower_body_image_{i}.jpg")
#         else:
#             print(f"Warning: lower_body_pred[{i}] 크기 {lower_body_pred[i].size}가 예상과 다릅니다. 스킵합니다.")

#         # 악세사리 예측이 있는 경우 처리
#         if accessory_pred is not None and len(accessory_pred) > i and accessory_pred[i].shape[0] > 0:
#             for j in range(min(accessory_pred.shape[1], 5)):  # 최대 5개의 악세사리 처리
#                 if accessory_pred[i][j].size == 30000:  # 벡터 크기 확인
#                     predicted_accessory_image = accessory_pred[i][j].reshape(100, 100, 3)
#                     predicted_accessory_image = np.clip(predicted_accessory_image, 0, 255).astype('uint8')
#                     cv2.imwrite(os.path.join(output_folder, f"predicted_accessory_image_{i}_{j}.jpg"), predicted_accessory_image)
#                     print(f"Saved: predicted_accessory_image_{i}_{j}.jpg")
#                 else:
#                     print(f"Warning: accessory_pred[{i}, {j}] 크기 {accessory_pred[i][j].size}가 예상과 다릅니다. 스킵합니다.")
# else:
#     print("Error: 예측된 벡터가 비어 있습니다.")

# print(f"예측된 이미지들이 폴더에 저장되었습니다: {output_folder}")


In [37]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# 학습된 모델 로드
model = load_model('pattern_color_model.h5')

# 저장할 폴더 경로 설정
output_folder = 'predicted_images'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)  # 폴더가 없으면 생성

# 유효성 검사용 데이터 준비
upper_body_data_valid = np.array(upper_body_data_valid)
accessory_data_valid_padded = np.array(
    [pad_sequence(seq, MAX_SEQUENCE_LENGTH, 30000) for seq in accessory_data_valid], dtype=np.float32
)

# 모델 예측
predictions = model.predict([upper_body_data_valid, accessory_data_valid_padded])

# 예측값을 확인하여 처리
if isinstance(predictions, list) and len(predictions) == 2:
    lower_body_pred = predictions[0]
    accessory_pred = predictions[1]
else:
    print("Error: 예측 결과가 예상한 형태가 아닙니다.")
    lower_body_pred = np.array([])
    accessory_pred = np.array([])

# 예측 벡터 크기 확인
print("Shape of lower_body_pred:", lower_body_pred.shape)
print("Shape of accessory_pred:", accessory_pred.shape)

# 예측 결과 처리 및 이미지 저장
if lower_body_pred.size > 0 and accessory_pred.size > 0:
    for i in range(min(5, len(lower_body_pred))):  # 처음 5개만 저장
        # 하의 예측 벡터 처리
        if lower_body_pred[i].size == 30000:  # 벡터 크기가 30000인지 확인
            try:
                predicted_lower_body_image = lower_body_pred[i].reshape(100, 100, 3)
                predicted_lower_body_image = np.clip(predicted_lower_body_image, 0, 255).astype('uint8')
                cv2.imwrite(os.path.join(output_folder, f"predicted_lower_body_image_{i}.jpg"), predicted_lower_body_image)
                print(f"Saved: predicted_lower_body_image_{i}.jpg")
            except Exception as e:
                print(f"Error saving lower body image {i}: {e}")
        else:
            print(f"Warning: lower_body_pred[{i}] 크기 {lower_body_pred[i].size}가 예상과 다릅니다. 스킵합니다.")

        # 악세사리 예측이 있는 경우 처리
        if accessory_pred is not None and len(accessory_pred) > i and accessory_pred[i].shape[0] > 0:
            for j in range(min(accessory_pred.shape[1], 5)):  # 최대 5개의 악세사리 처리
                if accessory_pred[i][j].size == 30000:  # 벡터 크기 확인
                    try:
                        predicted_accessory_image = accessory_pred[i][j].reshape(100, 100, 3)
                        predicted_accessory_image = np.clip(predicted_accessory_image, 0, 255).astype('uint8')
                        cv2.imwrite(os.path.join(output_folder, f"predicted_accessory_image_{i}_{j}.jpg"), predicted_accessory_image)
                        print(f"Saved: predicted_accessory_image_{i}_{j}.jpg")
                    except Exception as e:
                        print(f"Error saving accessory image {i}_{j}: {e}")
                else:
                    print(f"Warning: accessory_pred[{i}, {j}] 크기 {accessory_pred[i][j].size}가 예상과 다릅니다. 스킵합니다.")
else:
    print("Error: 예측된 벡터가 비어 있습니다.")

print(f"예측된 이미지들이 폴더에 저장되었습니다: {output_folder}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Error: 예측 결과가 예상한 형태가 아닙니다.
Shape of lower_body_pred: (0,)
Shape of accessory_pred: (0,)
Error: 예측된 벡터가 비어 있습니다.
예측된 이미지들이 폴더에 저장되었습니다: predicted_images


GG